In [1]:
'''
当前模型无法输出清晰的分界线，这里枚举了一些针对性措施
'''

'\n当前模型无法输出清晰的分界线，这里枚举了一些针对性措施\n'

In [2]:
'''
统一设置地址
'''

import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)

# 修改当前工作目录，以后输出文件只需要写文件名
new_dir = "D:/李娅宁/肩台外侧点-0715/"
os.chdir(new_dir)
print("修改后的工作目录：", os.getcwd())


当前工作目录： C:\Users\HP
修改后的工作目录： D:\李娅宁\肩台外侧点-0715


In [3]:
'''
sobel算子滤波
'''

'\nsobel算子滤波\n'

In [10]:
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 读取点云数据和标签
def load_labeled_point_cloud(file_path):
    data = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 4:
                x, y, z, label = map(float, parts)
                data.append([x, y, z])
                labels.append(int(label))
    return np.array(data), np.array(labels)

def create_voxel_grid(data, labels, grid_size):
    if data.size == 0 or labels.size == 0:
        print(f"Warning: Empty data or labels array. data.size={data.size}, labels.size={labels.size}")
        return None, None

    grid = np.zeros((grid_size, grid_size, grid_size))
    label_grid = np.zeros((grid_size, grid_size, grid_size))

    min_coords = np.min(data, axis=0)
    max_coords = np.max(data, axis=0)
    voxel_dim = (max_coords - min_coords) / grid_size

    for i, point in enumerate(data):
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, 0, grid_size-1)  # Ensure indices are within bounds
        grid[voxel[0], voxel[1], voxel[2]] = 1
        label_grid[voxel[0], voxel[1], voxel[2]] = labels[i] - 1  # Convert labels 1 and 2 to 0 and 1

    return grid, label_grid

def load_data_from_directory(data_dir, grid_size=16):
    x_data = []
    y_data = []
    for file in os.listdir(data_dir):
        if file.endswith("_labeled.txt"):
            file_path = os.path.join(data_dir, file)
            data, labels = load_labeled_point_cloud(file_path)
            if data.size == 0 or labels.size == 0:
                print(f"Skipping empty file: {file_path}")
                continue
            voxel_grid, label_grid = create_voxel_grid(data, labels, grid_size)
            if voxel_grid is not None and label_grid is not None:
                # 应用数据增强
                voxel_grid = augment_voxel_grid(voxel_grid)
                x_data.append(voxel_grid)
                y_data.append(label_grid)
    x_data = np.expand_dims(np.array(x_data), axis=-1)
    y_data = np.expand_dims(np.array(y_data), axis=-1)
    return x_data, y_data

# 数据增强
def augment_voxel_grid(voxel_grid):
    """
    对体素网格进行随机旋转和翻转。
    """
    # 随机旋转
    angle = np.random.uniform(0, 360)
    voxel_grid = np.rot90(voxel_grid, k=int(angle // 90), axes=(0, 1))
    
    # 随机翻转
    if np.random.rand() > 0.5:
        voxel_grid = np.flip(voxel_grid, axis=0)
    if np.random.rand() > 0.5:
        voxel_grid = np.flip(voxel_grid, axis=1)
    if np.random.rand() > 0.5:
        voxel_grid = np.flip(voxel_grid, axis=2)
    
    return voxel_grid


# 定义sobel层
import tensorflow as tf
from tensorflow.keras.layers import Layer

class SobelLayer(Layer):
    def __init__(self):
        super(SobelLayer, self).__init__()

    def call(self, inputs):
        # 3D Sobel kernels
        sobel_x = tf.constant([[[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
                                [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
                                [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]],

                               [[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
                                [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
                                [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]],

                               [[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
                                [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
                                [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]]], dtype=tf.float32)

        sobel_y = tf.constant([[[[-1, -2, -1], [-1, -2, -1], [-1, -2, -1]],
                                [[0, 0, 0], [0, 0, 0], [0, 0, 0]],
                                [[1, 2, 1], [1, 2, 1], [1, 2, 1]]],

                               [[[-1, -2, -1], [-1, -2, -1], [-1, -2, -1]],
                                [[0, 0, 0], [0, 0, 0], [0, 0, 0]],
                                [[1, 2, 1], [1, 2, 1], [1, 2, 1]]],

                               [[[-1, -2, -1], [-1, -2, -1], [-1, -2, -1]],
                                [[0, 0, 0], [0, 0, 0], [0, 0, 0]],
                                [[1, 2, 1], [1, 2, 1], [1, 2, 1]]]], dtype=tf.float32)

        sobel_z = tf.constant([[[[-1, -2, -1], [0, 0, 0], [1, 2, 1]],
                                [[-1, -2, -1], [0, 0, 0], [1, 2, 1]],
                                [[-1, -2, -1], [0, 0, 0], [1, 2, 1]]],

                               [[[-1, -2, -1], [0, 0, 0], [1, 2, 1]],
                                [[-1, -2, -1], [0, 0, 0], [1, 2, 1]],
                                [[-1, -2, -1], [0, 0, 0], [1, 2, 1]]],

                               [[[-1, -2, -1], [0, 0, 0], [1, 2, 1]],
                                [[-1, -2, -1], [0, 0, 0], [1, 2, 1]],
                                [[-1, -2, -1], [0, 0, 0], [1, 2, 1]]]], dtype=tf.float32)

        sobel_x = tf.reshape(sobel_x, [3, 3, 3, 2, 1])
        sobel_y = tf.reshape(sobel_y, [3, 3, 3, 2, 1])
        sobel_z = tf.reshape(sobel_z, [3, 3, 3, 2, 1])

        gx = tf.nn.conv3d(inputs, sobel_x, strides=[1, 1, 1, 1, 1], padding='SAME')
        gy = tf.nn.conv3d(inputs, sobel_y, strides=[1, 1, 1, 1, 1], padding='SAME')
        gz = tf.nn.conv3d(inputs, sobel_z, strides=[1, 1, 1, 1, 1], padding='SAME')

        sobel_magnitude = tf.sqrt(tf.square(gx) + tf.square(gy) + tf.square(gz))
        return sobel_magnitude

    def compute_output_shape(self, input_shape):
        return input_shape



# 定义UNET模型
def unet_3d(input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)
    print(f"Input shape: {inputs.shape}")

    # Encoder
    conv1 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(inputs)
    print(f"conv1 shape: {conv1.shape}")
    conv1 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv1)
    print(f"conv1 shape: {conv1.shape}")
    pool1 = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    print(f"pool1 shape: {pool1.shape}")

    conv2 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(pool1)
    print(f"conv2 shape: {conv2.shape}")
    conv2 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv2)
    print(f"conv2 shape: {conv2.shape}")
    pool2 = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    print(f"pool2 shape: {pool2.shape}")

    conv3 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(pool2)
    print(f"conv3 shape: {conv3.shape}")
    conv3 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv3)
    print(f"conv3 shape: {conv3.shape}")
    pool3 = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(conv3)
    print(f"pool3 shape: {pool3.shape}")

    conv4 = tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(pool3)
    print(f"conv4 shape: {conv4.shape}")
    conv4 = tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv4)
    print(f"conv4 shape: {conv4.shape}")
    pool4 = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(conv4)
    print(f"pool4 shape: {pool4.shape}")

    conv5 = tf.keras.layers.Conv3D(1024, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(pool4)
    print(f"conv5 shape: {conv5.shape}")
    conv5 = tf.keras.layers.Conv3D(1024, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv5)
    print(f"conv5 shape: {conv5.shape}")

    # Decoder
    up6 = tf.keras.layers.Conv3D(512, 2, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(tf.keras.layers.UpSampling3D(size=(2, 2, 2))(conv5))
    print(f"up6 shape: {up6.shape}")
    merge6 = tf.keras.layers.concatenate([conv4, up6], axis=4)
    print(f"merge6 shape: {merge6.shape}")
    conv6 = tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(merge6)
    print(f"conv6 shape: {conv6.shape}")
    conv6 = tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv6)
    print(f"conv6 shape: {conv6.shape}")

    up7 = tf.keras.layers.Conv3D(256, 2, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(tf.keras.layers.UpSampling3D(size=(2, 2, 2))(conv6))
    print(f"up7 shape: {up7.shape}")
    merge7 = tf.keras.layers.concatenate([conv3, up7], axis=4)
    print(f"merge7 shape: {merge7.shape}")
    conv7 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(merge7)
    print(f"conv7 shape: {conv7.shape}")
    conv7 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv7)
    print(f"conv7 shape: {conv7.shape}")

    up8 = tf.keras.layers.Conv3D(128, 2, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(tf.keras.layers.UpSampling3D(size=(2, 2, 2))(conv7))
    print(f"up8 shape: {up8.shape}")
    merge8 = tf.keras.layers.concatenate([conv2, up8], axis=4)
    print(f"merge8 shape: {merge8.shape}")
    conv8 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(merge8)
    print(f"conv8 shape: {conv8.shape}")
    conv8 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv8)
    print(f"conv8 shape: {conv8.shape}")

    up9 = tf.keras.layers.Conv3D(64, 2, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(tf.keras.layers.UpSampling3D(size=(2, 2, 2))(conv8))
    print(f"up9 shape: {up9.shape}")
    merge9 = tf.keras.layers.concatenate([conv1, up9], axis=4)
    print(f"merge9 shape: {merge9.shape}")
    conv9 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(merge9)
    print(f"conv9 shape: {conv9.shape}")
    conv9 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv9)
    print(f"conv9 shape: {conv9.shape}")
    conv9 = tf.keras.layers.Conv3D(2, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv9)
    print(f"conv9 shape: {conv9.shape}")

    # 在模型的末尾添加 Sobel 层
    sobel_layer = SobelLayer()(conv9)
    print(f"sobel_layer shape: {sobel_layer.shape}")

    conv10 = tf.keras.layers.Conv3D(1, 1, activation='sigmoid')(sobel_layer)
    print(f"conv10 shape: {conv10.shape}")

    model = tf.keras.Model(inputs, conv10)
    return model

# 设置数据路径
data_dir = '重新处理后的数据_手工筛选'

# 加载数据
x_data, y_data = load_data_from_directory(data_dir, grid_size=16)

# 分割数据集
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

# 定义学习率调度函数
def scheduler(epoch, lr):
    if epoch < 5:
        return float(lr)
    else:
        return float(lr * tf.math.exp(-0.2).numpy())

# 定义学习率调度回调和早停回调
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 定义模型
input_shape = (16, 16, 16, 1)
model = unet_3d(input_shape)

# 编译模型
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
history = model.fit(
    x_train, y_train, 
    validation_split=0.1,
    epochs=2, 
    batch_size=10,
    callbacks=[lr_scheduler, early_stopping]
)

# 评估模型
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# 绘制训练过程中的损失和精确度
def plot_training_history(history):
    """
    绘制训练过程中的损失和精确度。
    """
    # 绘制损失
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Over Epochs')
    plt.legend()

    # 绘制精确度
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.legend()

    plt.tight_layout()
    plt.show()

# 绘制训练历史
plot_training_history(history)

# 保存模型
model_save_path = 'July27模型/原始UNET_Sobel.h5'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

Input shape: (None, 16, 16, 16, 1)
conv1 shape: (None, 16, 16, 16, 64)
conv1 shape: (None, 16, 16, 16, 64)
pool1 shape: (None, 8, 8, 8, 64)
conv2 shape: (None, 8, 8, 8, 128)
conv2 shape: (None, 8, 8, 8, 128)
pool2 shape: (None, 4, 4, 4, 128)
conv3 shape: (None, 4, 4, 4, 256)
conv3 shape: (None, 4, 4, 4, 256)
pool3 shape: (None, 2, 2, 2, 256)
conv4 shape: (None, 2, 2, 2, 512)
conv4 shape: (None, 2, 2, 2, 512)
pool4 shape: (None, 1, 1, 1, 512)
conv5 shape: (None, 1, 1, 1, 1024)
conv5 shape: (None, 1, 1, 1, 1024)
up6 shape: (None, 2, 2, 2, 512)
merge6 shape: (None, 2, 2, 2, 1024)
conv6 shape: (None, 2, 2, 2, 512)
conv6 shape: (None, 2, 2, 2, 512)
up7 shape: (None, 4, 4, 4, 256)
merge7 shape: (None, 4, 4, 4, 512)
conv7 shape: (None, 4, 4, 4, 256)
conv7 shape: (None, 4, 4, 4, 256)
up8 shape: (None, 8, 8, 8, 128)
merge8 shape: (None, 8, 8, 8, 256)
conv8 shape: (None, 8, 8, 8, 128)
conv8 shape: (None, 8, 8, 8, 128)
up9 shape: (None, 16, 16, 16, 64)
merge9 shape: (None, 16, 16, 16, 128)
conv9 

ValueError: Exception encountered when calling SobelLayer.call().

[1mCannot reshape a tensor with 81 elements to shape [3,3,3,2,1] (54 elements) for '{{node functional_5_1/sobel_layer_6_1/Reshape}} = Reshape[T=DT_FLOAT, Tshape=DT_INT32](functional_5_1/sobel_layer_6_1/Const, functional_5_1/sobel_layer_6_1/Reshape/shape)' with input shapes: [3,3,3,3], [5] and with input tensors computed as partial shapes: input[1] = [3,3,3,2,1].[0m

Arguments received by SobelLayer.call():
  • inputs=tf.Tensor(shape=(None, 16, 16, 16, 2), dtype=float32)

In [ ]:
'''
模型用于预测，以及结果可视化
'''

# 开启交互旋转
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from scipy.spatial import KDTree
from collections import defaultdict
from itertools import combinations

# 加载obj文件
def load_obj_file(file_path):
    vertices = []
    faces = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = [float(parts[1]), float(parts[2]), float(parts[3])]
                    vertices.append(vertex)
                elif line.startswith('f '):
                    parts = line.strip().split()
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                    faces.append(face)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return vertices, faces

# 将顶点平移至包围盒中心
def center_vertices(vertices):
    vertices_array = np.array(vertices)
    min_coords = vertices_array.min(axis=0)
    max_coords = vertices_array.max(axis=0)
    center = (min_coords + max_coords) / 2
    centered_vertices = vertices_array - center
    return centered_vertices.tolist()

# 点云转体素网格
def create_voxel_grid(data, grid_size):
    grid = np.zeros((grid_size, grid_size, grid_size))
    min_coords = np.min(data, axis=0)
    max_coords = np.max(data, axis=0)
    voxel_dim = (max_coords - min_coords) / grid_size

    for i, point in enumerate(data):
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, 0, grid_size-1)  # Ensure indices are within bounds
        grid[voxel[0], voxel[1], voxel[2]] = 1

    return grid, min_coords, voxel_dim

# 旋转点云
def rotate_points(points, angles):
    x_angle, y_angle, z_angle = angles
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(x_angle), -np.sin(x_angle)],
        [0, np.sin(x_angle), np.cos(x_angle)]
    ])
    Ry = np.array([
        [np.cos(y_angle), 0, np.sin(y_angle)],
        [0, 1, 0],
        [-np.sin(y_angle), 0, np.cos(y_angle)]
    ])
    Rz = np.array([
        [np.cos(z_angle), -np.sin(z_angle), 0],
        [np.sin(z_angle), np.cos(z_angle), 0],
        [0, 0, 1]
    ])
    R = Rz @ Ry @ Rx
    return points @ R.T

# 从训练好的模型获取标签
def get_labels_from_model(model, voxel_grid):
    voxel_grid = np.expand_dims(voxel_grid, axis=0)  # Add batch dimension
    voxel_grid = np.expand_dims(voxel_grid, axis=-1)  # Add channel dimension
    predictions = model.predict(voxel_grid)
    labels = (predictions > 0.5).astype(int)
    return labels.reshape(voxel_grid.shape[1], voxel_grid.shape[2], voxel_grid.shape[3])

# 应用预测标签到原始点云
def apply_labels_to_point_cloud(data, predicted_labels, min_coords, voxel_dim, grid_size):
    labels = np.zeros(len(data))
    for i, point in enumerate(data):
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, 0, grid_size-1)  # Ensure indices are within bounds
        labels[i] = predicted_labels[voxel[0], voxel[1], voxel[2]]
    return labels

# 识别并合并分割线段
def find_boundary_edges(vertices, faces, labels):
    edges = defaultdict(list)
    for face in faces:
        for (v1, v2) in combinations(face, 2):
            if labels[v1] != labels[v2]:
                edges[tuple(sorted([v1, v2]))].append(face)
    
    # Create an adjacency list for edges
    adjacency_list = defaultdict(list)
    for (v1, v2), faces in edges.items():
        adjacency_list[v1].append(v2)
        adjacency_list[v2].append(v1)
    
    # Find all connected components of edges
    visited = set()
    boundary_lines = []
    for vertex in adjacency_list:
        if vertex not in visited:
            stack = [vertex]
            boundary_line = []
            while stack:
                current = stack.pop()
                if current not in visited:
                    visited.add(current)
                    boundary_line.append(current)
                    for neighbor in adjacency_list[current]:
                        if neighbor not in visited:
                            stack.append(neighbor)
            boundary_lines.append(boundary_line)
    
    # Calculate the length of each boundary line and sort by length
    boundary_lines = sorted(boundary_lines, key=lambda line: sum(np.linalg.norm(vertices[line[i]] - vertices[line[i + 1]]) for i in range(len(line) - 1)), reverse=True)
    return boundary_lines

# 绘制带有分类标签的点云和分界线
def plot_surface_with_marks(vertices, faces, labels, part1, part2, view_angles=(30, 30), angles=(0, 0, 0)):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    vertices = np.array(vertices)
    faces = np.array(faces)

    # Apply rotation
    vertices = rotate_points(vertices, angles)
    
    x, y, z = vertices.T

    try:
        # 标记分割的点云部分
        part1_faces = [face for face in faces if sum(labels[vertex] == 0 for vertex in face) > 1]
        part2_faces = [face for face in faces if sum(labels[vertex] == 1 for vertex in face) > 1]
        
        part1_faces = np.array(part1_faces)
        part2_faces = np.array(part2_faces)
        
        if len(part1_faces) > 0:
            ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=part1_faces, color='cornflowerblue', alpha=0.6)
        if len(part2_faces) > 0:
            ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=part2_faces, color='honeydew', alpha=0.6)

        # 找到边界线并绘制最长的边界线
        boundary_lines = find_boundary_edges(vertices, faces, labels)
        if boundary_lines:
            longest_boundary_line = boundary_lines[0]
            ax.plot(vertices[longest_boundary_line, 0], vertices[longest_boundary_line, 1], vertices[longest_boundary_line, 2], color='red')
            
    except ValueError as e:
        print(f"ValueError: {e}")
        return

    # 设置标签和标题
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_title('3D Model with Segmentation and Boundary Lines')

    # 确保坐标轴刻度一致
    max_range = np.array([max(x)-min(x), max(y)-min(y), max(z)-min(z)]).max()
    mid_x = (max(x) + min(x)) * 0.5
    mid_y = (max(y) + min(y)) * 0.5
    mid_z = (max(z) + min(z)) * 0.5
    ax.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
    ax.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
    ax.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

    # 设置视角
    elev, azim = view_angles
    ax.view_init(elev=elev, azim=azim)  # Adjust these values as needed

    # 确保坐标轴比例相等
    ax.set_box_aspect([1,1,1])  # Aspect ratio is 1:1:1

    # 启用交互式旋转
    plt.show()

# 加载点云数据
obj_file_path = r'D:/李娅宁/肩台外侧点-0715/已完成预处理的原始数据/15/15_1.obj'
vertices, faces = load_obj_file(obj_file_path)
centered_vertices = center_vertices(vertices)

# 将点云转换为体素网格
grid_size = 16  # Ensure grid size matches model requirements
voxel_grid, min_coords, voxel_dim = create_voxel_grid(np.array(centered_vertices), grid_size)

# 使用训练好的模型进行预测
predicted_labels = get_labels_from_model(model, voxel_grid)

# 获取原始点云的预测标签
predicted_point_labels = apply_labels_to_point_cloud(np.array(centered_vertices), predicted_labels, min_coords, voxel_dim, grid_size)

# 定义分割的部分（需要根据实际情况定义）
part1 = {i for i, label in enumerate(predicted_point_labels) if label == 0}
part2 = {i for i, label in enumerate(predicted_point_labels) if label == 1}

# 画图
plot_surface_with_marks(centered_vertices, faces, predicted_point_labels, part1, part2, view_angles=(30, 30), angles=(np.radians(90), np.radians(-30), np.radians(30)))


In [ ]:
boundary损失函数在这里适用吗？